In [1]:
from approx.approximate import ApprOXimate
from approx.feature_engineering import MaterialFeatureExtractor
from mendeleev.fetch import fetch_table

In [2]:
import pandas as pd

df = pd.read_csv('featurised_data.csv')

df = df.fillna(0)
df = df.drop(columns=['Compound', 'Lowest distortion', 'formula'])
df

,μ,all_valence_s,all_valence_p,all_valence_d,all_valence_f,all_valence_total,all_unfilled_valence_s,all_unfilled_valence_p,all_unfilled_valence_d,all_unfilled_valence_f,...,var_valence_s,var_valence_p,var_valence_d,var_valence_f,var_valence_total,var_unfilled_valence_s,var_unfilled_valence_p,var_unfilled_valence_d,var_unfilled_valence_f,var_unfilled_valence_total
0,0.800000,1.2,2.4,0.000000,0.0,3.600000,0.8,3.6,10.000000,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.678571,1.2,2.4,1.800000,0.0,5.400000,0.8,3.6,8.200000,14.0,...,0.0,0.0,9.0,0.0,9.0,2.0,6.0,1.0,14.0,23.0
2,0.385714,1.2,2.4,0.000000,0.0,3.600000,0.8,3.6,10.000000,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.371429,1.6,2.4,2.000000,0.0,6.000000,0.4,3.6,8.000000,14.0,...,2.0,0.0,10.0,0.0,12.0,0.0,6.0,0.0,14.0,20.0
4,0.664286,1.2,2.4,1.600000,0.0,5.200000,0.8,3.6,8.400000,14.0,...,0.0,0.0,8.0,0.0,8.0,2.0,6.0,2.0,14.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5324,0.442857,1.0,2.0,0.666667,0.0,3.666667,1.0,4.0,9.333333,14.0,...,0.0,0.0,2.0,0.0,2.0,2.0,6.0,8.0,14.0,30.0
5325,0.642857,1.2,2.4,0.000000,0.0,3.600000,0.8,3.6,10.000000,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5326,0.678571,1.2,2.4,0.000000,0.0,3.600000,0.8,3.6,10.000000,14.0,...,0.0,0.0,0.0,0.0,0.0,2.0,6.0,10.0,14.0,32.0
5327,0.528571,1.2,2.4,2.000000,0.0,5.600000,0.8,3.6,8.000000,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# df["Lowest distortion"] = df["Lowest distortion"].astype("category")
# df["Lowest_distortion_code"] = df["Lowest distortion"].cat.codes

In [5]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['μ']) 
y = df['μ']

# test split
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42)

# train / val split
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val,
    test_size=0.20 / 0.90,
    random_state=42)

In [6]:
from time import time
import pandas as pd

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error

from sklearn.dummy import DummyRegressor

from sklearn.linear_model import Ridge

from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.neighbors import KNeighborsRegressor

from sklearn.svm import SVR
from sklearn.svm import LinearSVR

def instantiate_model(model_name):
    model = model_name()
    return model

def fit_model(model, X_train, y_train):
    ti = time()
    model = instantiate_model(model)
    model.fit(X_train, y_train)
    fit_time = time() - ti
    return model, fit_time

def evaluate_model(model, X, y_act):
    y_pred = model.predict(X)
    r2 = r2_score(y_act, y_pred)
    mae = mean_absolute_error(y_act, y_pred)
    rmse_val = root_mean_squared_error(y_act, y_pred)
    return r2, mae, rmse_val

def fit_evaluate_model(model, model_name, X_train, y_train, X_val, y_act_val):
    model, fit_time = fit_model(model, X_train, y_train)
    r2_train, mae_train, rmse_train = evaluate_model(model, X_train, y_train)
    r2_val, mae_val, rmse_val = evaluate_model(model, X_val, y_act_val)
    result_dict = {
        'model_name': model_name,
        'model_name_pretty': type(model).__name__,
        'model_params': model.get_params(),
        'fit_time': fit_time,
        'r2_train': r2_train,
        'mae_train': mae_train,
        'rmse_train': rmse_train,
        'r2_val': r2_val,
        'mae_val': mae_val,
        'rmse_val': rmse_val}
    return model, result_dict

def append_result_df(df, result_dict):
    df_result_appended = pd.concat([df, pd.DataFrame([result_dict])], ignore_index=True)
    return df_result_appended

def append_model_dict(dic, model_name, model):
    dic[model_name] = model
    return dic

In [7]:
from collections import OrderedDict

df_classics = pd.DataFrame(columns=['model_name',
                                    'model_name_pretty',
                                    'model_params',
                                    'fit_time',
                                    'r2_train',
                                    'mae_train',
                                    'rmse_train',
                                    'r2_val',
                                    'mae_val',
                                    'rmse_val'])

# Build a dictionary of model names
classic_model_names = OrderedDict({
    'dumr': DummyRegressor,
    'rr': Ridge,
    'abr': AdaBoostRegressor,
    'gbr': GradientBoostingRegressor,
    'rfr': RandomForestRegressor,
    'etr': ExtraTreesRegressor,
    'svr': SVR,
    'lsvr': LinearSVR,
    'knr': KNeighborsRegressor,
})

# Instantiate a dictionary to store the model objects
classic_models = OrderedDict()

# Keep track of elapsed time
ti = time()

# Loop through each model type, fit and predict, and evaluate and store results
for model_name, model in classic_model_names.items():
    print(f'Now fitting and evaluating model {model_name}: {model.__name__}')
    model, result_dict = fit_evaluate_model(model, model_name, X_train, y_train, X_val, y_val)
    df_classics = append_result_df(df_classics, result_dict)
    classic_models = append_model_dict(classic_models, model_name, model)

dt = time() - ti
print(f'Finished fitting {len(classic_models)} models, total time: {dt:0.2f} s')

Now fitting and evaluating model dumr: DummyRegressor
Now fitting and evaluating model rr: Ridge
Now fitting and evaluating model abr: AdaBoostRegressor
Now fitting and evaluating model gbr: GradientBoostingRegressor


C:\Users\fayat\AppData\Local\Temp\ipykernel_32832\1087646447.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_result_appended = pd.concat([df, pd.DataFrame([result_dict])], ignore_index=True)


Now fitting and evaluating model rfr: RandomForestRegressor
Now fitting and evaluating model etr: ExtraTreesRegressor
Now fitting and evaluating model svr: SVR
Now fitting and evaluating model lsvr: LinearSVR


c:\Users\fayat\Documents\apprOXimate\.venv\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Now fitting and evaluating model knr: KNeighborsRegressor
Finished fitting 9 models, total time: 12.15 s


In [8]:
df_classics = df_classics.sort_values('r2_val', ignore_index=True)
df_classics

,model_name,model_name_pretty,model_params,fit_time,r2_train,mae_train,rmse_train,r2_val,mae_val,rmse_val
0,etr,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",2.084650,0.674415,0.063508,0.101090,-0.598682,0.166114,0.226177
1,rfr,RandomForestRegressor,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",4.768716,0.647038,0.075608,0.105254,-0.121726,0.141154,0.189457
2,dumr,DummyRegressor,"{'constant': None, 'quantile': None, 'strategy...",0.000718,0.000000,0.139426,0.177163,-0.000013,0.140359,0.178884
3,knr,KNeighborsRegressor,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.002000,0.517956,0.092923,0.123003,0.269022,0.114684,0.152940
4,gbr,GradientBoostingRegressor,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ...",1.366971,0.516879,0.095388,0.123141,0.346941,0.108190,0.144559
5,abr,AdaBoostRegressor,"{'estimator': None, 'learning_rate': 1.0, 'los...",0.178346,0.433823,0.105589,0.133306,0.389602,0.109819,0.139757
6,lsvr,LinearSVR,"{'C': 1.0, 'dual': 'auto', 'epsilon': 0.0, 'fi...",0.252081,0.421817,0.104776,0.134712,0.395869,0.107081,0.139038
7,svr,SVR,"{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'd...",0.310584,0.445334,0.102361,0.131944,0.420478,0.105086,0.136177
8,rr,Ridge,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...",0.007653,0.445635,0.102229,0.131908,0.420940,0.105337,0.136122
